In [27]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.services import Service
from selenium.webdriver.chrome.options import Options
import matplotlib.pyplot as plt 
import os
import logging


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Correct path to chromedriver
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")

# Optional: Maximize the window
driver.maximize_window()


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Launch Chrome with the correct chromedriver path
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")

# Maximize window (optional)
driver.maximize_window()

# ✅ Navigate to the correct Wikipedia page
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")


In [9]:
from bs4 import BeautifulSoup

# Step 1: Get the page source from the open browser
html = driver.page_source

# Step 2: Parse with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Step 3: Find all list items under main content
# (20th century events are in <ul><li> format under content)
events = soup.select("div.mw-parser-output > ul > li")

# Step 4: Extract text and print
for i, event in enumerate(events[:20]):  # just showing first 20 for demo
    print(f"{i+1}. {event.get_text()}")


1. Timeline of the 20th century
2. Infectious disease in the 20th century
3. Death rates in the 20th century
4. List of battles 1901-2000
5. 20th century inventions
6. 20th-century art
7. List of 20th-century religious leaders
8. Modernism
9. Modern art
10. UNESCO (28 February 2008). "The Twentieth Century". History of Humanity. Vol. VII. Routledge. p. 600. ISBN 978-0-415-09311-8.
11. The 20th Century Research Project (archived 26 February 2012)
12. Slouching Towards Utopia: The Economic History of the Twentieth Century (archived 6 February 2012)
13. TIME Archives The greatest writers of the 20th Century (archived 17 January 2005)


In [28]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Set up Selenium
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")
time.sleep(3)

# Parse HTML
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Grab all <li> items inside <ul>s (these usually have the years)
events = soup.select("div.mw-parser-output ul > li")

# Filter out anything that’s too short or not event-like
filtered_events = [li.get_text() for li in events if len(li.get_text()) > 20]

# Save to file
with open("20th_century_events.txt", "w", encoding="utf-8") as f:
    for i, event in enumerate(filtered_events[:100]):  # limiting to 100 for demo
        f.write(f"{i+1}. {event}\n")

print("✅ Saved the events with years to '20th_century_events.txt'")
driver.quit()


✅ Saved the events with years to '20th_century_events.txt'


In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Launch browser and go to the Wikipedia page
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")
time.sleep(3)

# Grab page source
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Close the browser now that we have the HTML
driver.quit()

# Find all year-based headers and the lists under them
output = []

for header in soup.select("h3"):
    decade = header.text.strip()
    next_node = header.find_next_sibling()

    while next_node and next_node.name != "h3":
        if next_node.name == "ul":
            for li in next_node.find_all("li"):
                event = li.get_text(strip=True)
                output.append(f"{decade} - {event}")
        next_node = next_node.find_next_sibling()

# Save to .txt file
with open("20th_century_events.txt", "w", encoding="utf-8") as f:
    for i, item in enumerate(output[:100]):  # limit to 100 for now
        f.write(f"{i+1}. {item}\n")

print("✅ Saved cleaned timeline events with years.")


✅ Saved cleaned timeline events with years.


In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Open Chrome and go to the Wikipedia page
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Wait for page to fully load
time.sleep(3)

# Step 3: Get page HTML and parse with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Extract actual timeline content (decade + events)
output = []

# Loop through all decades (e.g., h3 with 'span.mw-headline')
for heading in soup.select("h3"):
    span = heading.select_one("span.mw-headline")
    if span:
        decade = span.get_text(strip=True)

        # Find the next <ul> that holds events
        next_el = heading.find_next_sibling()
        while next_el and next_el.name != "h3":
            if next_el.name == "ul":
                for li in next_el.find_all("li"):
                    event = li.get_text(strip=True)
                    output.append(f"{decade} - {event}")
            next_el = next_el.find_next_sibling()

# Step 5: Preview first 20 results
for i, item in enumerate(output[:20], start=1):
    print(f"{i}. {item}")

# 🛑 Do NOT quit the driver yet — wait for you to inspect it


In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Let the page fully load
time.sleep(3)

# Step 3: Scrape page source and parse with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Find timeline data under headings and bullet lists
output = []
headings = soup.find_all(["h2", "h3"])

for heading in headings:
    if heading.find("span", class_="mw-headline"):
        decade = heading.get_text(strip=True).replace("[edit]", "")

        # Look ahead to the next sibling elements
        next_el = heading.find_next_sibling()
        while next_el and next_el.name not in ["h2", "h3"]:
            if next_el.name == "ul":
                for li in next_el.find_all("li"):
                    event = li.get_text(strip=True)
                    output.append(f"{decade} - {event}")
            next_el = next_el.find_next_sibling()

# Step 5: Preview first 20
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")


In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Let the page fully load
time.sleep(3)

# Step 3: Scrape page source and parse with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Try to capture headings and events
output = []
headings = soup.find_all(["h2", "h3"])
print(f"Found {len(headings)} headings")

for heading in headings:
    if heading.find("span", class_="mw-headline"):
        decade = heading.get_text(strip=True).replace("[edit]", "")
        print(f"\n🔍 Checking heading: {decade}")

        # Move to next sibling
        next_el = heading.find_next_sibling()
        while next_el and next_el.name not in ["h2", "h3"]:
            print(f"➡️ Next sibling tag: {next_el.name}")
            if next_el.name == "ul":
                for li in next_el.find_all("li"):
                    event = li.get_text(strip=True)
                    output.append(f"{decade} – {event}")
            next_el = next_el.find_next_sibling()

# Step 5: Preview first 20 results
print("\n📋 First 20 extracted events:")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")


Found 12 headings

📋 First 20 extracted events:


In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Let the page fully load
time.sleep(3)

# Step 3: Get page source and parse
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Extract events under each heading
output = []
headings = soup.find_all(["h2", "h3"])
print(f"Found {len(headings)} headings\n")

for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue
    section = span.get_text(strip=True).replace("[edit]", "")
    print(f"🔍 Checking heading: {section}")

    next_el = heading.find_next_sibling()
    while next_el and next_el.name not in ["h2", "h3"]:
        print(f"   ➡️ Next sibling tag: {next_el.name}")
        if next_el.name == "ul":
            for li in next_el.find_all("li"):
                event = li.get_text(strip=True)
                output.append(f"{section} – {event}")
        next_el = next_el.find_next_sibling()

# Step 5: Show result
print("\n📋 First 20 extracted events:")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")


Found 12 headings


📋 First 20 extracted events:


In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser and load page
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")
time.sleep(3)

# Step 2: Parse with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 3: Loop through headings and find nearby <li> items
output = []
headings = soup.find_all(["h2", "h3"])
print(f"Found {len(headings)} headings")

for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue
    section = span.get_text(strip=True).replace("[edit]", "")
    print(f"\n🔍 Heading: {section}")

    sibling = heading.find_next_sibling()
    while sibling and sibling.name not in ["h2", "h3"]:
        lis = sibling.find_all("li")
        if lis:
            print(f"   ➡️ Found {len(lis)} list items")
        for li in lis:
            event = li.get_text(strip=True)
            output.append(f"{section} – {event}")
        sibling = sibling.find_next_sibling()

# Step 4: Print results
print("\n📋 First 20 extracted events:")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")


Found 12 headings

📋 First 20 extracted events:


In [17]:
for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue
    section = span.get_text(strip=True).replace("[edit]", "")
    print(f"\n🔍 Heading: {section}")

    sibling = heading.find_next_sibling()
    while sibling and sibling.name not in ["h2", "h3"]:
        print(f"➡️ Checking sibling tag: {sibling.name}")
        
        lis = sibling.find_all("li")
        print(f"   ➕ Found {len(lis)} list items")

        for li in lis:
            event = li.get_text(strip=True)
            output.append(f"{section} – {event}")

        sibling = sibling.find_next_sibling()


In [18]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Let the page fully load
time.sleep(3)

# Step 3: Scrape page source and parse with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Loop through headings and dig through nested content
output = []
headings = soup.find_all(["h2", "h3"])
print(f"Found {len(headings)} headings")

for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue

    section = span.get_text(strip=True).replace("[edit]", "")
    print(f"\n🔍 Heading: {section}")

    sibling = heading.find_next_sibling()
    while sibling and sibling.name not in ["h2", "h3"]:
        # Check inside any <ul> found in this sibling
        uls = sibling.find_all("ul")
        for ul in uls:
            lis = ul.find_all("li")
            print(f"➡️ {section} – Found {len(lis)} list items")
            for li in lis:
                event = li.get_text(strip=True)
                output.append(f"{section} – {event}")
        
        sibling = sibling.find_next_sibling()

# Step 5: Print the output
print("\n📋 First 20 extracted events:")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")


Found 12 headings

📋 First 20 extracted events:


In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser and go to the page
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Let the page fully load
time.sleep(3)

# Step 3: Parse HTML with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Get all text from the page
text = soup.get_text()
text = text.encode('utf-8')

# Step 5: Save to .txt file
with open('20th_century_events.txt', 'wb') as f:
    f.write(text)

# Optional: Check the title to confirm you got the right page
print(soup.title)

# Close browser after saving
driver.quit()


<title>Key events of the 20th century - Wikipedia</title>


In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

# Step 2: Wait for page to load
time.sleep(3)

# Step 3: Get page content
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 4: Extract section headers and bullet events
output = []

headings = soup.find_all(["h2", "h3"])

for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue
    section = span.get_text(strip=True).replace("[edit]", "")
    
    # Skip boring sections
    if "See also" in section or "References" in section or "External links" in section:
        break

    sibling = heading.find_next_sibling()
    
    while sibling and sibling.name not in ["h2", "h3"]:
        if sibling.name == "ul":
            for li in sibling.find_all("li"):
                event = li.get_text(strip=True)
                output.append(f"{section} – {event}")
        sibling = sibling.find_next_sibling()

# Step 5: Print first 20 results to confirm
print("\n📄 First 20 extracted events:\n")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")



📄 First 20 extracted events:



In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch Chrome
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")
time.sleep(3)

# Step 2: Parse the HTML
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 3: Extract timeline data
output = []
headings = soup.find_all(["h2", "h3"])

for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue
    
    section = span.get_text(strip=True)
    
    # Skip irrelevant sections
    if section in ["See also", "References", "External links", "Navigation menu"]:
        break

    # Traverse through siblings until a new heading is found
    sibling = heading.find_next_sibling()
    while sibling and sibling.name not in ["h2", "h3"]:
        if sibling.name in ["ul", "ol"]:
            items = sibling.find_all("li")
            for li in items:
                event = li.get_text(strip=True)
                output.append(f"{section} – {event}")
        sibling = sibling.find_next_sibling()

# Step 4: Show output
print(f"\n📄 Found {len(output)} events\n")
print("📌 First 20 extracted events:\n")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")



📄 Found 0 events

📌 First 20 extracted events:



In [22]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch browser
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Timeline_of_the_20th_century")
time.sleep(3)

# Step 2: Scrape content
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 3: Parse year headings and events
output = []
headings = soup.find_all("h3")

for heading in headings:
    span = heading.find("span", class_="mw-headline")
    if not span:
        continue
    year = span.get_text(strip=True)

    # Now check for events in the next <ul>
    next_el = heading.find_next_sibling()
    while next_el and next_el.name != "h3":
        if next_el.name == "ul":
            for li in next_el.find_all("li"):
                event = li.get_text(strip=True)
                output.append(f"{year} – {event}")
        next_el = next_el.find_next_sibling()

# Step 4: Show results
print(f"\n📄 Found {len(output)} events")
print("📌 First 20 extracted events:\n")
for i, item in enumerate(output[:20], 1):
    print(f"{i}. {item}")



📄 Found 0 events
📌 First 20 extracted events:



In [23]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Step 1: Launch Chrome and go to the page
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver")
driver.get("https://en.wikipedia.org/wiki/Timeline_of_the_20th_century")
time.sleep(3)

# Step 2: Get HTML and parse it
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Step 3: Extract only text with bullets (•)
text = soup.get_text()
lines = text.split("\n")
events = [line.strip() for line in lines if line.strip().startswith("•")]

# Step 4: Show output
print(f"\n📄 Found {len(events)} events with bullet points")
print("📌 First 20 extracted events:\n")
for i, event in enumerate(events[:20], 1):
    print(f"{i}. {event}")



📄 Found 0 events with bullet points
📌 First 20 extracted events:



In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Step 1: Launch browser and open the correct Wikipedia page
driver = webdriver.Chrome(executable_path='/opt/homebrew/bin/chromedriver')
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")
time.sleep(3)

# Step 2: Find all sections with class 'div-col' (these contain grouped events)
sections = driver.find_elements(by=By.CLASS_NAME, value="div-col")

# Step 3: Combine text from each div-col element
all_events = []
for section in sections:
    text = section.text.strip().split("\n")
    all_events.extend(text)

# Step 4: Print results
print(f"\n📌 Found {len(all_events)} events in div-col sections.\n")
for i, event in enumerate(all_events[:20], 1):
    print(f"{i}. {event}")



📌 Found 0 events in div-col sections.



In [29]:
df = pd.DataFrame(list_char, columns = ["character"])

NameError: name 'pd' is not defined

In [31]:
import pandas as pd


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [32]:
import pandas as pd
pd.__version__


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [33]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [34]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject